# Submitit tutorial
See `https://github.com/fairinternal/submitit` for more information and examples.

Install with `pip install git+ssh://git@github.com/fairinternal/submitit@master#egg=submitit`

**Contributors** (chronological): Jérémy Rapin, Louis Martin, Lowik Chanussot, Lucas Hosseini, Fabio Petroni, Francisco Massa, Guillaume Wenzek, Thibaut Lavril, Vinayak Tantia **(come join us :D)**

Jupyter needs to be run from a conda env with `submitit` installed. This tutorial works on both Slurm and Chronos.

### This is the function we will run *from a devfair* on slurm:

In [ ]:
import time

def wait(duration, error=False):
    for k in range(duration):
        time.sleep(1)
        print(f"Waited {k + 1}s.", flush=True)
    if error:
        raise ValueError("Sh*t happens")
    return 12

In [ ]:
wait(3)  # prints every second and returns 12

In [ ]:
wait(0, True)  # raises an error

## Example 1 - Basics of submitit

In [ ]:
import submitit
import time  # used in most examples

The following cell makes the whole tutorial compatible with both Slurm and Chronos

In [ ]:
import os
chronos_cpu = "fblearner_ash_cpuram_default"  # this is the equivalent of a partition for Chronos
log_folder = "logs/demo/%j"
if submitit.AutoExecutor.which() == "chronos":  # needs to be on a shared folder for Chronos
    log_folder = f"/mnt/vol/gfsai-east/ai-group/users/{os.environ['USER']}/{folder}"

Executors are the objects that submit jobs (same API for Slurm and Chronos), the folder is used for logging:

In [ ]:
executor = submitit.AutoExecutor(folder=log_folder)

Configure the executor (basically what would have gone into your sbatch file):

In [ ]:
executor.update_parameters(timeout_min=3, partition="dev", constraint="pascal", hostgroup=chronos_cpu)

Then just submit the function and the arguments to the cluster (Eg.: `wait(10)`), and access all information:

In [ ]:
job = executor.submit(wait, 10)

In [ ]:
job

In [ ]:
print("id", job.job_id)
print("Finished?", job.done())
print("state", job.state)

In [ ]:
print(job.stdout())

In [ ]:
job.result()  # waits for result if not completed, and returns it

Under the hood, files are created for logging, submission and results:

In [ ]:
list(job.paths.folder.glob("*"))

## Example 2 - Error handling

In [ ]:
job = executor.submit(wait, 0, error=True)

In [ ]:
job.result()

In [ ]:
print(job.stderr())

In [ ]:
err = job.exception()  # returns None if the job finished and did not raised an error

**Note: prefer submitting module defined functions instead of script defined functions for better traceback and robustness**

## Example 3 - Checkpointing

 1. use a callable object instead of a function, so that it keeps track of its state.
 2. add a `checkpoint` method, it will be called a checkpoint time with same parameters as `__call__`.
 3. make it return a `DelayedSubmission` object, with the same syntax as in `executor.submit`.
 4. Profit.
 
 **Note**: This example only works on Slurm, because timed-out jobs are not requeued with Chronos (ideas

In [ ]:
class Wait:

    def __init__(self):
        self.index = 0  # this is the state of your "function"

    def __call__(self, duration):
        for self.index in range(self.index, duration):
            time.sleep(1)
            print(f"Waited {self.index + 1}s.", flush=True)
        return 12

    def checkpoint(self, duration):  # same parameter as __call__
        func = self  # self is an instance of Wait, like func = Wait()
        return submitit.helpers.DelayedSubmission(func, duration)  # same parameters as executor.submit

In [ ]:
executor = submitit.AutoExecutor(folder=log_folder, max_num_timeout=3, warn_ignored=False)
executor.update_parameters(partition="dev", timeout_min=2, constraint="pascal", hostgroup=chronos_cpu)

In [ ]:
func = Wait()

In [ ]:
jobp = executor.submit(func, 300)

In [ ]:
print(jobp)

In [ ]:
print(jobp.stdout())

In [ ]:
jobp._send_requeue_signal(timeout=False)  # you could preempt manually for testing

## Example 4 - Multitask jobs

In [ ]:
def printer():
    jobenv = submitit.JobEnvironment()
    print(jobenv, flush=True)
    return jobenv.local_rank

In [ ]:
executor.update_parameters(tasks_per_node=3, nodes=2)

In [ ]:
job = executor.submit(printer)

In [ ]:
job

In [ ]:
print(job.stdout())  # concatenate all logs and returns

In [ ]:
print(job.task(4).stdout())  # get only task 4 logs

In [ ]:
# use results() instead of result() for correct type hints in multi-task jobs
job.results()  

In [ ]:
print(job.task(4).result())  # get only task 4 return value

**Note: the job is requeued an unlimited number of times if preempted, but is limited to `max_num_timeout` timeouts**

## Example 5 (optional) - Same interface as concurrent.futures

In [ ]:
executor = submitit.AutoExecutor(folder=log_folder, max_num_timeout=3, warn_ignored=False)
executor.update_parameters(timeout_min=1, partition="dev", constraint="pascal", hostgroup=chronos_cpu)

In [ ]:
jobs = [executor.submit(wait, duration=k) for k in range(2, 7)]

In [ ]:
num_finished = sum(j.done() for j in jobs)
print(f"{num_finished} finished jobs")  # probably around 2 have finished, given the overhead

Concurrent.futures jobs and executors work exactly the same way:

In [ ]:
from concurrent import futures
with futures.ThreadPoolExecutor(max_workers=10) as tp_executor:  # This is the only real difference!
    tp_jobs = [tp_executor.submit(wait, duration=k) for k in range(2, 7)]

See `https://github.com/fairinternal/submitit` for more information.

Install with `pip install git+ssh://git@github.com/fairinternal/submitit@master#egg=submitit`

Contributors: Jérémy Rapin, Louis Martin, Lowik Chanussot, Lucas Hosseini (come join us :D)

## Example 6 (optional) - Local Executor

In [ ]:
# You can work with a local folder, or with a shared folder (eg /checkpoint)
local_folder = os.path.expandvars("/scratch/$USER/submitit_tutorial/%j")

In [ ]:
executor = submitit.LocalExecutor(folder=local_folder)
# Only timeout_min, tasks_per_node and gpus_per_node are taken into account
executor.update_parameters(timeout_min=1, tasks_per_node=3) #, gpus_per_node=1)

In [ ]:
# Same as Wait, but use Checkpointable and print the JobEnvironment
class WaitMultiTask(submitit.helpers.Checkpointable):
    def __init__(self):
        self.index = 0  # this is the state of your "function"

    def __call__(self, duration):
        jobenv = submitit.JobEnvironment()
        print(jobenv, flush=True)
        for self.index in range(self.index, duration):
            time.sleep(1)
            if (self.index + 1) % 10 == 0:
                print(f"Waited {self.index + 1}s.", flush=True)
        return jobenv.global_rank

In [ ]:
job1 = executor.submit(WaitMultiTask(), duration=30) # This one will finish before timeout
job2 = executor.submit(WaitMultiTask(), duration=90) # This one will timeout, then requeue and finish

In [ ]:
# Results should be the same
print(f"Job 1 results: {job1.result()}")
print(f"Job 2 results: {job2.result()}")

In [ ]:
# get only task 2 logs
print(job1.task(0).stdout()) 

In [ ]:
# Look for requeue messages in stdout
print(job2.task(0).stdout())

In [ ]:
# Note that only the master (global_rank=0) does checkpointing
print(job2.task(1).stdout())

## Example 7 - Job arrays on FAIR cluster (`executor.map_array`)

On slurm, job arrays have multiple advantages compared to standard jobs:
- uses only 1 call to slurm (avoids flooding it).
- faster to submit (for the same reason).
- lets you define a cap on how many jobs can run in parallel at any given time, so you can send thousands of jobs without breaking the scheduler, as long as you leave a reasonable value for this parallelism.


Job arrays can be used through `executor.map_array` in `submitit`, and the parallelism is managed the the `array_parallelism` parameter.

**Note**: while the method works on both clusters, those advantages are currently only relevant for FAIR cluster.

In [ ]:
def addition(a, b):
    return a + b

In [ ]:
# define list of parameters, map_array will act as zipping each list and submitting to the function
a = [1, 2, 3, 4]
b = [10, 20, 30, 40]

In [ ]:
executor = submitit.AutoExecutor(folder=log_folder)
# the following line tells the scheduler to only run
# at most 2 jobs at once. By default, this is several hundreds
executor.update_parameters(array_parallelism=2)
jobs = executor.map_array(addition, a, b)
print(jobs)  # notice job id: this is now "<array_job_id>_<array_task_id>" with task id from 0 to 3

In [ ]:
assert tuple(j.result() for j in jobs) == (11, 22, 33, 44)  # a[0] + b[0] = 11, etc...
